In [3]:
import gradio as gr
import numpy as np
import random
import platform

from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO

model = YOLO('yolov8n.pt')


def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def get_font():
    font_size = 20
    if platform.system() == "Darwin":
        font = ImageFont.truetype("AppleGothic.ttf", size=font_size)
    elif platform.system() == "Windows":
        font = ImageFont.truetype("malgun.ttf", size=font_size)
    else:
        font = ImageFont.load_default(size=font_size)
    
    return font


def detect_objects(image):
    
    drawn_image = Image.fromarray(image.copy())
    draw = ImageDraw.Draw(drawn_image)
    
    # 이미지의 height, width 를 뽑아낸다.
    height , width = image.shape[:2]
    print(height, width)

    response = model(image)

    for output in response:
        bounding_box_list = output.boxes.xyxy.cpu().numpy()
        confidence_list = output.boxes.conf.cpu().numpy()
        label_list = output.names

        for bounding_box, confidence, label_index in zip(bounding_box_list, confidence_list, label_list):
            x1, y1, x2, y2 = bounding_box
            label = label_list[label_index]

            color = random_color()

            draw.rectangle((x1, y1, x2, y2), outline = color, width=3)
            draw.text((x1+5, y1+5), text="{} : {:.2f}%".format(label, confidence*100), fill="red", font=get_font())

    return drawn_image

with gr.Blocks() as demo:
    
    def stream_webcam(image):
        # detect function.
        drawn_image = detect_objects(image)
        return drawn_image
    
    webcam_input = gr.Image(label="실시간 화면", sources="webcam", width=480, height=270, mirror_webcam=False)
    output_image = gr.Image(label="검출 화면", type="pil")
    
    webcam_input.stream(stream_webcam, inputs=[webcam_input], outputs=[output_image])
    
demo.launch()
# image = cv2.imread("/Users/fimtrus/Pictures/901.기타/taj-mahal-3654227_1280.jpg")
# detect_objects(image)

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.
